In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels)) # real loss
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # minimize Q
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # maximize Q
    eloss += tf.reduce_mean(targetQs) # minimize Q
    aloss2 += -tf.reduce_mean(targetQs) # maxizmie Q
    eloss += tf.reduce_mean(eQs_logits) # minimize Q
    aloss2 += -tf.reduce_mean(eQs_logits) # maxizmie Q
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:0.7002 eloss:4.3245 aloss2:2.1319 exploreP:0.9980
Episode:1 meanR:19.0000 R:18.0000 rate:0.0360 aloss:0.6982 eloss:4.2298 aloss2:2.2813 exploreP:0.9962
Episode:2 meanR:21.3333 R:26.0000 rate:0.0520 aloss:0.6998 eloss:4.2404 aloss2:2.2709 exploreP:0.9937
Episode:3 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.6963 eloss:4.2152 aloss2:2.3142 exploreP:0.9921
Episode:4 meanR:20.2000 R:21.0000 rate:0.0420 aloss:0.6949 eloss:4.1910 aloss2:2.3700 exploreP:0.9901
Episode:5 meanR:18.8333 R:12.0000 rate:0.0240 aloss:0.6974 eloss:4.1858 aloss2:2.3647 exploreP:0.9889
Episode:6 meanR:17.8571 R:12.0000 rate:0.0240 aloss:0.6924 eloss:4.1757 aloss2:2.3931 exploreP:0.9877
Episode:7 meanR:17.8750 R:18.0000 rate:0.0360 aloss:0.6935 eloss:4.1896 aloss2:2.3623 exploreP:0.9859
Episode:8 meanR:17.3333 R:13.0000 rate:0.0260 aloss:0.6978 eloss:4.1411 aloss2:2.4381 exploreP:0.9847
Episode:9 meanR:19.1000 R:35.0000 rate:0.0700 aloss:0.6937 eloss:4.1232 aloss2:2.5

Episode:80 meanR:25.0247 R:20.0000 rate:0.0400 aloss:0.6837 eloss:3.8232 aloss2:3.9466 exploreP:0.8184
Episode:81 meanR:25.3659 R:53.0000 rate:0.1060 aloss:0.6788 eloss:3.7743 aloss2:3.7820 exploreP:0.8141
Episode:82 meanR:25.1807 R:10.0000 rate:0.0200 aloss:0.6826 eloss:3.7672 aloss2:3.9091 exploreP:0.8133
Episode:83 meanR:25.1071 R:19.0000 rate:0.0380 aloss:0.6777 eloss:3.7668 aloss2:3.8231 exploreP:0.8118
Episode:84 meanR:25.0000 R:16.0000 rate:0.0320 aloss:0.7000 eloss:3.7956 aloss2:3.8544 exploreP:0.8105
Episode:85 meanR:25.1860 R:41.0000 rate:0.0820 aloss:0.6838 eloss:3.7731 aloss2:3.8896 exploreP:0.8072
Episode:86 meanR:25.1724 R:24.0000 rate:0.0480 aloss:0.6992 eloss:3.8384 aloss2:3.9398 exploreP:0.8053
Episode:87 meanR:25.0000 R:10.0000 rate:0.0200 aloss:0.7007 eloss:3.7668 aloss2:3.8308 exploreP:0.8045
Episode:88 meanR:25.0112 R:26.0000 rate:0.0520 aloss:0.6868 eloss:3.7648 aloss2:3.8131 exploreP:0.8024
Episode:89 meanR:25.1111 R:34.0000 rate:0.0680 aloss:0.6934 eloss:3.7681 

Episode:159 meanR:37.9800 R:14.0000 rate:0.0280 aloss:0.7729 eloss:3.7703 aloss2:3.8739 exploreP:0.5963
Episode:160 meanR:38.8800 R:106.0000 rate:0.2120 aloss:0.7243 eloss:3.7439 aloss2:3.9212 exploreP:0.5901
Episode:161 meanR:38.9200 R:37.0000 rate:0.0740 aloss:0.7172 eloss:3.7370 aloss2:3.9527 exploreP:0.5880
Episode:162 meanR:39.0900 R:44.0000 rate:0.0880 aloss:0.7423 eloss:3.7403 aloss2:3.9642 exploreP:0.5854
Episode:163 meanR:39.2500 R:38.0000 rate:0.0760 aloss:0.7279 eloss:3.7414 aloss2:3.9404 exploreP:0.5832
Episode:164 meanR:39.3400 R:28.0000 rate:0.0560 aloss:0.6991 eloss:3.7414 aloss2:3.9478 exploreP:0.5816
Episode:165 meanR:39.2600 R:33.0000 rate:0.0660 aloss:0.7208 eloss:3.7373 aloss2:3.9396 exploreP:0.5798
Episode:166 meanR:39.6100 R:50.0000 rate:0.1000 aloss:0.7398 eloss:3.7446 aloss2:3.9521 exploreP:0.5769
Episode:167 meanR:39.8200 R:35.0000 rate:0.0700 aloss:0.7164 eloss:3.7612 aloss2:3.9364 exploreP:0.5749
Episode:168 meanR:40.0300 R:49.0000 rate:0.0980 aloss:0.7076 el

Episode:238 meanR:40.8900 R:39.0000 rate:0.0780 aloss:0.7435 eloss:3.7641 aloss2:3.9416 exploreP:0.4359
Episode:239 meanR:40.4400 R:65.0000 rate:0.1300 aloss:0.7320 eloss:3.7555 aloss2:3.9473 exploreP:0.4331
Episode:240 meanR:40.7400 R:42.0000 rate:0.0840 aloss:0.7305 eloss:3.7514 aloss2:3.9580 exploreP:0.4314
Episode:241 meanR:40.4200 R:45.0000 rate:0.0900 aloss:0.7241 eloss:3.7444 aloss2:3.9755 exploreP:0.4295
Episode:242 meanR:40.6800 R:73.0000 rate:0.1460 aloss:0.7331 eloss:3.7514 aloss2:3.9642 exploreP:0.4264
Episode:243 meanR:40.6600 R:28.0000 rate:0.0560 aloss:0.7082 eloss:3.7355 aloss2:3.9728 exploreP:0.4253
Episode:244 meanR:40.6300 R:50.0000 rate:0.1000 aloss:0.7334 eloss:3.7682 aloss2:3.9627 exploreP:0.4232
Episode:245 meanR:40.8200 R:44.0000 rate:0.0880 aloss:0.7559 eloss:3.7448 aloss2:3.9621 exploreP:0.4214
Episode:246 meanR:41.0400 R:47.0000 rate:0.0940 aloss:0.7589 eloss:3.7392 aloss2:3.9771 exploreP:0.4194
Episode:247 meanR:40.7900 R:39.0000 rate:0.0780 aloss:0.7605 elo

Episode:317 meanR:43.1800 R:36.0000 rate:0.0720 aloss:0.7955 eloss:3.7585 aloss2:3.9370 exploreP:0.3102
Episode:318 meanR:43.1900 R:33.0000 rate:0.0660 aloss:0.7571 eloss:3.7619 aloss2:3.9296 exploreP:0.3093
Episode:319 meanR:43.4600 R:54.0000 rate:0.1080 aloss:0.7403 eloss:3.7487 aloss2:3.9465 exploreP:0.3076
Episode:320 meanR:43.4100 R:45.0000 rate:0.0900 aloss:0.7599 eloss:3.7552 aloss2:3.9338 exploreP:0.3063
Episode:321 meanR:43.5100 R:46.0000 rate:0.0920 aloss:0.7729 eloss:3.7568 aloss2:3.9562 exploreP:0.3049
Episode:322 meanR:43.8500 R:54.0000 rate:0.1080 aloss:0.7368 eloss:3.7465 aloss2:3.9325 exploreP:0.3034
Episode:323 meanR:43.7200 R:34.0000 rate:0.0680 aloss:0.7443 eloss:3.7416 aloss2:3.9489 exploreP:0.3024
Episode:324 meanR:43.5300 R:32.0000 rate:0.0640 aloss:0.7349 eloss:3.7466 aloss2:3.9504 exploreP:0.3014
Episode:325 meanR:43.9200 R:80.0000 rate:0.1600 aloss:0.7405 eloss:3.7487 aloss2:3.9468 exploreP:0.2991
Episode:326 meanR:43.7200 R:38.0000 rate:0.0760 aloss:0.7463 elo

Episode:396 meanR:46.8200 R:40.0000 rate:0.0800 aloss:0.7107 eloss:3.7679 aloss2:3.9366 exploreP:0.2180
Episode:397 meanR:46.9900 R:44.0000 rate:0.0880 aloss:0.7155 eloss:3.7303 aloss2:3.9109 exploreP:0.2171
Episode:398 meanR:46.8800 R:32.0000 rate:0.0640 aloss:0.7382 eloss:3.7305 aloss2:3.9256 exploreP:0.2164
Episode:399 meanR:46.8500 R:38.0000 rate:0.0760 aloss:0.7346 eloss:3.7351 aloss2:3.9334 exploreP:0.2157
Episode:400 meanR:46.8600 R:38.0000 rate:0.0760 aloss:0.7315 eloss:3.7732 aloss2:3.9332 exploreP:0.2149
Episode:401 meanR:46.6600 R:40.0000 rate:0.0800 aloss:0.7166 eloss:3.7272 aloss2:3.9092 exploreP:0.2141
Episode:402 meanR:46.8400 R:51.0000 rate:0.1020 aloss:0.7179 eloss:3.7222 aloss2:3.9268 exploreP:0.2130
Episode:403 meanR:46.7700 R:38.0000 rate:0.0760 aloss:0.7666 eloss:3.7355 aloss2:3.9293 exploreP:0.2123
Episode:404 meanR:46.9000 R:54.0000 rate:0.1080 aloss:0.7046 eloss:3.7270 aloss2:3.9294 exploreP:0.2112
Episode:405 meanR:46.7500 R:30.0000 rate:0.0600 aloss:0.7515 elo

Episode:475 meanR:52.5600 R:95.0000 rate:0.1900 aloss:0.6777 eloss:3.7234 aloss2:3.9279 exploreP:0.1459
Episode:476 meanR:52.5800 R:56.0000 rate:0.1120 aloss:0.6868 eloss:3.7129 aloss2:3.9460 exploreP:0.1451
Episode:477 meanR:52.8400 R:71.0000 rate:0.1420 aloss:0.6846 eloss:3.7213 aloss2:3.9376 exploreP:0.1442
Episode:478 meanR:53.4300 R:103.0000 rate:0.2060 aloss:0.6864 eloss:3.7104 aloss2:3.9256 exploreP:0.1428
Episode:479 meanR:53.5100 R:68.0000 rate:0.1360 aloss:0.7009 eloss:3.7248 aloss2:3.9291 exploreP:0.1419
Episode:480 meanR:53.8900 R:66.0000 rate:0.1320 aloss:0.6791 eloss:3.7172 aloss2:3.9315 exploreP:0.1410
Episode:481 meanR:54.0300 R:80.0000 rate:0.1600 aloss:0.6987 eloss:3.7281 aloss2:3.9398 exploreP:0.1400
Episode:482 meanR:54.0100 R:44.0000 rate:0.0880 aloss:0.6881 eloss:3.7081 aloss2:3.9220 exploreP:0.1394
Episode:483 meanR:54.1900 R:54.0000 rate:0.1080 aloss:0.7002 eloss:3.7328 aloss2:3.9089 exploreP:0.1387
Episode:484 meanR:54.3500 R:76.0000 rate:0.1520 aloss:0.6905 el

Episode:554 meanR:81.5800 R:112.0000 rate:0.2240 aloss:0.6767 eloss:3.7012 aloss2:4.0338 exploreP:0.0801
Episode:555 meanR:82.7300 R:158.0000 rate:0.3160 aloss:0.6796 eloss:3.6995 aloss2:4.0236 exploreP:0.0790
Episode:556 meanR:83.3400 R:111.0000 rate:0.2220 aloss:0.6756 eloss:3.6954 aloss2:4.0360 exploreP:0.0782
Episode:557 meanR:84.0900 R:112.0000 rate:0.2240 aloss:0.6817 eloss:3.7045 aloss2:4.0311 exploreP:0.0775
Episode:558 meanR:83.9000 R:42.0000 rate:0.0840 aloss:0.6651 eloss:3.6909 aloss2:4.0272 exploreP:0.0772
Episode:559 meanR:84.6700 R:115.0000 rate:0.2300 aloss:0.6595 eloss:3.6948 aloss2:4.0371 exploreP:0.0764
Episode:560 meanR:85.4000 R:120.0000 rate:0.2400 aloss:0.6753 eloss:3.6975 aloss2:4.0393 exploreP:0.0756
Episode:561 meanR:84.8300 R:127.0000 rate:0.2540 aloss:0.6771 eloss:3.6883 aloss2:4.0418 exploreP:0.0748
Episode:562 meanR:85.0600 R:92.0000 rate:0.1840 aloss:0.6706 eloss:3.7126 aloss2:4.0241 exploreP:0.0742
Episode:563 meanR:86.0000 R:154.0000 rate:0.3080 aloss:0.

Episode:632 meanR:134.5900 R:126.0000 rate:0.2520 aloss:0.6548 eloss:3.6768 aloss2:4.2703 exploreP:0.0330
Episode:633 meanR:135.1500 R:195.0000 rate:0.3900 aloss:0.6374 eloss:3.6831 aloss2:4.2281 exploreP:0.0325
Episode:634 meanR:135.7000 R:129.0000 rate:0.2580 aloss:0.6477 eloss:3.6723 aloss2:4.2127 exploreP:0.0322
Episode:635 meanR:136.6000 R:157.0000 rate:0.3140 aloss:0.6403 eloss:3.6705 aloss2:4.2378 exploreP:0.0319
Episode:636 meanR:136.8400 R:163.0000 rate:0.3260 aloss:0.6494 eloss:3.6741 aloss2:4.2609 exploreP:0.0315
Episode:637 meanR:137.4900 R:186.0000 rate:0.3720 aloss:0.6539 eloss:3.6702 aloss2:4.2753 exploreP:0.0311
Episode:638 meanR:137.9100 R:111.0000 rate:0.2220 aloss:0.6245 eloss:3.6778 aloss2:4.2573 exploreP:0.0309
Episode:639 meanR:138.6200 R:138.0000 rate:0.2760 aloss:0.6669 eloss:3.6647 aloss2:4.2866 exploreP:0.0306
Episode:640 meanR:139.0300 R:142.0000 rate:0.2840 aloss:0.6550 eloss:3.6567 aloss2:4.2816 exploreP:0.0303
Episode:641 meanR:138.9700 R:115.0000 rate:0.2

Episode:710 meanR:146.1900 R:184.0000 rate:0.3680 aloss:0.5926 eloss:3.6726 aloss2:4.3433 exploreP:0.0175
Episode:711 meanR:146.0400 R:147.0000 rate:0.2940 aloss:0.6190 eloss:3.6629 aloss2:4.3917 exploreP:0.0174
Episode:712 meanR:145.5100 R:127.0000 rate:0.2540 aloss:0.5953 eloss:3.6837 aloss2:4.3905 exploreP:0.0173
Episode:713 meanR:145.4000 R:146.0000 rate:0.2920 aloss:0.6195 eloss:3.6737 aloss2:4.3547 exploreP:0.0172
Episode:714 meanR:145.5800 R:177.0000 rate:0.3540 aloss:0.5942 eloss:3.6725 aloss2:4.3790 exploreP:0.0170
Episode:715 meanR:146.2300 R:206.0000 rate:0.4120 aloss:0.6191 eloss:3.6643 aloss2:4.3744 exploreP:0.0169
Episode:716 meanR:146.3700 R:159.0000 rate:0.3180 aloss:0.6065 eloss:3.6696 aloss2:4.3466 exploreP:0.0168
Episode:717 meanR:146.3200 R:155.0000 rate:0.3100 aloss:0.6122 eloss:3.6695 aloss2:4.3556 exploreP:0.0167
Episode:718 meanR:145.9500 R:137.0000 rate:0.2740 aloss:0.6034 eloss:3.6649 aloss2:4.3783 exploreP:0.0166
Episode:719 meanR:145.4000 R:133.0000 rate:0.2

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
